<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/MENA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/2015.csv',encoding = "ISO-8859-1", engine = 'python')
data

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176
...,...,...,...,...,...,...,...,...,...,...,...,...
153,Rwanda,Sub-Saharan Africa,154,3.465,0.03464,0.22208,0.77370,0.42864,0.59201,0.55191,0.22628,0.67042
154,Benin,Sub-Saharan Africa,155,3.340,0.03656,0.28665,0.35386,0.31910,0.48450,0.08010,0.18260,1.63328
155,Syria,Middle East and Northern Africa,156,3.006,0.05015,0.66320,0.47489,0.72193,0.15684,0.18906,0.47179,0.32858
156,Burundi,Sub-Saharan Africa,157,2.905,0.08658,0.01530,0.41587,0.22396,0.11850,0.10062,0.19727,1.83302


In [174]:
Region_nums = {"Region":     {'Western Europe':0, 'North America':1, 'Australia and New Zealand':2,
 'Middle East and Northern Africa':3, 'Latin America and Caribbean':4,
 'Southeastern Asia':5, 'Central and Eastern Europe':9, 'Eastern Asia':6,
 'Sub-Saharan Africa':7, 'Southern Asia':8}}
print(type(Region_nums))
data['Region'].replace(Region_nums, inplace=True)
Countries=[]
Countries=data['Country'].unique()
Countries={Countries[i]: i for i in range(len(Countries))}
Countries_nums = Countries
print(Countries_nums)
data['Country'].replace(Countries_nums, inplace=True)
data['Country']
data

<class 'dict'>
{'Switzerland': 0, 'Iceland': 1, 'Denmark': 2, 'Norway': 3, 'Canada': 4, 'Finland': 5, 'Netherlands': 6, 'Sweden': 7, 'New Zealand': 8, 'Australia': 9, 'Israel': 10, 'Costa Rica': 11, 'Austria': 12, 'Mexico': 13, 'United States': 14, 'Brazil': 15, 'Luxembourg': 16, 'Ireland': 17, 'Belgium': 18, 'United Arab Emirates': 19, 'United Kingdom': 20, 'Oman': 21, 'Venezuela': 22, 'Singapore': 23, 'Panama': 24, 'Germany': 25, 'Chile': 26, 'Qatar': 27, 'France': 28, 'Argentina': 29, 'Czech Republic': 30, 'Uruguay': 31, 'Colombia': 32, 'Thailand': 33, 'Saudi Arabia': 34, 'Spain': 35, 'Malta': 36, 'Taiwan': 37, 'Kuwait': 38, 'Suriname': 39, 'Trinidad and Tobago': 40, 'El Salvador': 41, 'Guatemala': 42, 'Uzbekistan': 43, 'Slovakia': 44, 'Japan': 45, 'South Korea': 46, 'Ecuador': 47, 'Bahrain': 48, 'Italy': 49, 'Bolivia': 50, 'Moldova': 51, 'Paraguay': 52, 'Kazakhstan': 53, 'Slovenia': 54, 'Lithuania': 55, 'Nicaragua': 56, 'Peru': 57, 'Belarus': 58, 'Poland': 59, 'Malaysia': 60, 'Croa

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,0,0,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,1,0,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,2,0,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,3,0,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,4,1,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176
...,...,...,...,...,...,...,...,...,...,...,...,...
153,153,7,154,3.465,0.03464,0.22208,0.77370,0.42864,0.59201,0.55191,0.22628,0.67042
154,154,7,155,3.340,0.03656,0.28665,0.35386,0.31910,0.48450,0.08010,0.18260,1.63328
155,155,3,156,3.006,0.05015,0.66320,0.47489,0.72193,0.15684,0.18906,0.47179,0.32858
156,156,7,157,2.905,0.08658,0.01530,0.41587,0.22396,0.11850,0.10062,0.19727,1.83302


In [187]:
data.columns

Index(['Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'],
      dtype='object')

##Feature importance gives you a score for each feature of your data, the higher the score more important or relevant is the feature towards your output variable.Apply SelectKBest class to extract  features

In [227]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#apply SelectKBest class to extract top 10 best features
scaler = MinMaxScaler()
scaler.fit(data)
magic_norm = scaler.transform(data)
xx = magic_norm[:, 0:12]  # independent columns
xx = np.array(xx)

nameOflabels = [
 'Country', 'Region', 'Happiness Rank', 'Happiness Score',
       'Standard Error', 'Economy (GDP per Capita)', 'Family',
       'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)',
       'Generosity', 'Dystopia Residual'
]
xx=xx.transpose()
print(xx.shape)
df = pd.DataFrame(xx, index=nameOflabels)
xx=df.transpose()

yy = magic_norm[:, -1]  # target column
xx.shape
yy.shape

(12, 158)


(158,)

In [232]:
xx

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,0.000000,0.000000,0.000000,1.000000,0.131954,0.826132,0.962403,0.918244,0.993789,0.760595,0.372895,0.668630
1,0.006369,0.000000,0.006369,0.994524,0.256311,0.770412,1.000000,0.924496,0.938841,0.256292,0.548198,0.725030
2,0.012739,0.000000,0.012739,0.987363,0.124947,0.784113,0.970297,0.853099,0.969615,0.876175,0.428947,0.660889
3,0.019108,0.000000,0.019108,0.986310,0.171549,0.863099,0.949167,0.863409,1.000000,0.661394,0.435983,0.652724
4,0.025478,0.111111,0.025478,0.966302,0.143943,0.784592,0.943219,0.883326,0.945112,0.597144,0.575602,0.648584
...,...,...,...,...,...,...,...,...,...,...,...,...
153,0.974522,0.777778,0.974522,0.131845,0.136429,0.131376,0.551764,0.418083,0.883953,1.000000,0.284314,0.104425
154,0.980892,0.777778,0.980892,0.105518,0.152638,0.169573,0.252355,0.311241,0.723426,0.145132,0.229432,0.398557
155,0.987261,0.333333,0.987261,0.035173,0.267370,0.392329,0.338668,0.704150,0.234184,0.342556,0.592790,0.000000
156,0.993631,0.777778,0.993631,0.013901,0.574926,0.009051,0.296578,0.218444,0.176937,0.182312,0.247864,0.459573


In [0]:
import pandas
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(xx, yy)
# summarize scores
numpy.set_printoptions(precision=3)
print(fit.scores_)
features = fit.transform(xx)
# summarize selected features
print(features[0:5,:])